In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ategvyj4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ategvyj4
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12718 sha256=b906715994c2ce9982487e2c85564bcb0e56fa80d7327daff2cf4d9a5808ca54
  Stored in directory: /tmp/pip-ephem-wheel-cache-z0uql62j/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-06-13 05:01:26--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-06-13 05:01:26--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca4a46c7b02feff46a3c4c61805.dl.dropboxusercontent.com/cd/0/get/BQVqsHVxYrqN_NzqMziEHmC3hq9QeAyrHfPNa-Ew4GbYk1N4sEY5sfQUHcF7YxmcC3pP4EyciWOxyS-qPcLzgY_1N_5U93-Uy66E_uHxGExcksAbjVOtsPangCZBIzwND7n8MP05xNNREx53rouSdvIt/file?dl=1# [following]
--2021-06-13 05:01:27--  https://uca4a46c7b02feff46a3c4c61805.dl.dropboxusercontent.com/cd/0/get/BQVqsHVxYrqN_NzqMziEHmC3hq9QeAyrHfPNa-Ew4GbYk1N4sEY5sfQUHcF7YxmcC3pP4Eyci

In [ ]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6630269289016724 train acc 0.59375
epoch 1 batch id 201 loss 0.6711152791976929 train acc 0.5197450248756219
epoch 1 batch id 401 loss 0.3501186966896057 train acc 0.6238310473815462
epoch 1 batch id 601 loss 0.42186957597732544 train acc 0.68016846921797
epoch 1 batch id 801 loss 0.562883198261261 train acc 0.7137952559300874
epoch 1 batch id 1001 loss 0.5622569918632507 train acc 0.7366695804195804
epoch 1 batch id 1201 loss 0.5820509195327759 train acc 0.7531484179850125
epoch 1 batch id 1401 loss 0.33621785044670105 train acc 0.7651900428265525
epoch 1 batch id 1601 loss 0.5218673944473267 train acc 0.7743207370393504
epoch 1 batch id 1801 loss 0.539829432964325 train acc 0.7813888117712382
epoch 1 batch id 2001 loss 0.4080124497413635 train acc 0.7870752123938031
epoch 1 batch id 2201 loss 0.7233043313026428 train acc 0.792878237164925
epoch 1 batch id 2401 loss 0.3338404893875122 train acc 0.7977014785506039
epoch 1 batch id 2601 loss 0.2855648398399353 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.46289893617021277


...


epoch 2 test acc 0.42898936170212765


epoch 3 batch id 1 loss 0.5008273720741272 train acc 0.75
epoch 3 batch id 201 loss 0.19991672039031982 train acc 0.9149564676616916
epoch 3 batch id 401 loss 0.17151378095149994 train acc 0.9176278054862843
epoch 3 batch id 601 loss 0.1201326847076416 train acc 0.9185732113144759
epoch 3 batch id 801 loss 0.32318052649497986 train acc 0.9210752184769039
epoch 3 batch id 1001 loss 0.19968797266483307 train acc 0.9223276723276723
epoch 3 batch id 1201 loss 0.49761584401130676 train acc 0.9233451290591174
epoch 3 batch id 1401 loss 0.32486486434936523 train acc 0.9246297287651677
epoch 3 batch id 1601 loss 0.32158154249191284 train acc 0.9252810743285447
epoch 3 batch id 1801 loss 0.22138527035713196 train acc 0.9266553303720155
epoch 3 batch id 2001 loss 0.22352953255176544 train acc 0.927848575712144
epoch 3 batch id 2201 loss 0.41457265615463257 train acc 0.9289527487505679
epoch 3 batch id 2401 loss 0.35234591364860535 train acc 0.9299120158267389
epoch 3 batch id 2601 loss 0.0651622


epoch 3 test acc 0.469813829787234


epoch 4 batch id 1 loss 0.6005428433418274 train acc 0.84375
epoch 4 batch id 201 loss 0.0488491989672184 train acc 0.9532027363184079
epoch 4 batch id 401 loss 0.09002517908811569 train acc 0.9553460099750624
epoch 4 batch id 601 loss 0.027711885049939156 train acc 0.9558548252911814
epoch 4 batch id 801 loss 0.2707841396331787 train acc 0.955914481897628
epoch 4 batch id 1001 loss 0.31723955273628235 train acc 0.9572614885114885
epoch 4 batch id 1201 loss 0.4886166453361511 train acc 0.9579256869275604
epoch 4 batch id 1401 loss 0.04805177450180054 train acc 0.9585786937901499
epoch 4 batch id 1601 loss 0.25149625539779663 train acc 0.9593027795128045
epoch 4 batch id 1801 loss 0.17383907735347748 train acc 0.9596925319267073
epoch 4 batch id 2001 loss 0.03723156079649925 train acc 0.9604416541729135
epoch 4 batch id 2201 loss 0.21635879576206207 train acc 0.9610546342571559
epoch 4 batch id 2401 loss 0.035171497613191605 train acc 0.9614093086214077
epoch 4 batch id 2601 loss 0.0101


epoch 4 test acc 0.46914893617021275


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv(open('/content/drive/My Drive/텍마팀플/DAJUNG_comments.csv','rU'), encoding='utf-8', engine='c')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


In [ ]:
df

,comments,name,time,likes,topic
0,늘 영상만 보다가 처음 댓글 남겨요~^^ 큐앤에이 보니까 더 정감가고 친근하네요 다...,맨사언니,2021-04-04T14:15:48Z,1.0,NaN
1,"Hi eonni, give english subtitles please🙇",anggi naberia,2021-03-16T22:58:45Z,0.0,NaN
2,강아지 오둥이들 키우는 개아범 입니다 ~~~ 항상 유튜브 보면서 힐링하고 있어요~~...,설랜드네 티브이,2021-03-15T19:46:06Z,1.0,NaN
3,라이브못봐서 영상 기다렸어요,조서현,2021-03-13T08:59:54Z,1.0,NaN
4,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0.0,1.0
...,...,...,...,...,...
6312,도어알람귀여워요🥰 알콩달콩,박은경,2019-12-19T07:49:07Z,1.0,NaN
6313,"오, 세상 참 좋아졌네요 ㅎㅎ<br>스마트한 집이라니..",드림크루의 Hotspot,2019-12-19T05:06:38Z,2.0,NaN
6314,집이 너무 예뻐요~!! 조명 덕분에 더 분위기 있어 보이네요💕,예정효,2019-12-19T05:02:05Z,2.0,NaN
6315,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요!,김대원,2019-12-19T04:32:38Z,0.0,1.0


In [ ]:
import re

# html 태그 없애기
cleanr =re.compile('<.*?>')

#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

# 한글 외 영어, 숫자 등 제거
han = re.compile(r'[^가-힣 ]')

In [ ]:
# 전처리
cleantext = []
for i in df['comments']:
  text = re.sub(cleanr, "", i)
  text = re.sub(emoji_pattern," ",text)
  text = re.sub(han,"",text)
  cleantext.append(text)

In [ ]:
data = pd.DataFrame(cleantext, columns=['cleantext'])
data['label'] = 1
data.head()

,cleantext,label
0,늘 영상만 보다가 처음 댓글 남겨요 큐앤에이 보니까 더 정감가고 친근하네요 다정님,1
1,,1
2,강아지 오둥이들 키우는 개아범 입니다 항상 유튜브 보면서 힐링하고 있어요 가끔 집...,1
3,라이브못봐서 영상 기다렸어요,1
4,공대생이셨다니 멋져요 다음 라이브는 꼭 시간 맞춰올게요 아숩지뭐에요,1


In [ ]:
data.to_csv("/content/drive/My Drive/kobert/DAJUNG_댓글.txt", sep='\t', index=False)
dataset_test = nlp.data.TSVDataset("/content/drive/My Drive/kobert/DAJUNG_댓글.txt", field_indices=[0,1], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, 128, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=32, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
model.eval()
def model_pred(arg_list):
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    valid_length = valid_length
    segment_ids = segment_ids.long().to(device)
    label = label.long().to(device)
    pred_var = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(pred_var,1)
    arg_list.extend(predict.tolist())

list_pred = []
model_pred(list_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data["label"] = list_pred
data

,cleantext,label
0,늘 영상만 보다가 처음 댓글 남겨요 큐앤에이 보니까 더 정감가고 친근하네요 다정님,1
1,,0
2,강아지 오둥이들 키우는 개아범 입니다 항상 유튜브 보면서 힐링하고 있어요 가끔 집...,1
3,라이브못봐서 영상 기다렸어요,1
4,공대생이셨다니 멋져요 다음 라이브는 꼭 시간 맞춰올게요 아숩지뭐에요,1
...,...,...
6312,도어알람귀여워요 알콩달콩,1
6313,오 세상 참 좋아졌네요 스마트한 집이라니,1
6314,집이 너무 예뻐요 조명 덕분에 더 분위기 있어 보이네요,1
6315,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요,1


In [ ]:
data["label"] = list_pred
print(data.groupby("label").size())

label
0    1931
1    4386
dtype: int64


In [ ]:
# 부정
data[data["label"]==0]

,cleantext,label
1,,0
5,코 세척 비액 가루 넣어서 하는거 이상민이 하잖아요 그거 하시면 엄청 도움되요,0
6,절대 무섭지 않아요 한번하면 매일 하고 싶어요 개운함에 그리고 코 감기도 덜 걸리는...,0
9,브레드 캐이스는 어디서 구입하셨나요,0
23,캬,0
...,...,...
6303,양현서 허브 통해서 켜고 끄고 할 수 있어요 밝기 조절도 되구요,0
6304,얼음정수기인가요,0
6305,정수기는 얼음정수기는 아니고 코웨이 시루직수 정수기 입니다,0
6306,리엔케이 정수기는 얼음정수기는 아니고 코웨이 시루직수 정수기 입니다,0


In [ ]:
# 긍정
data[data["label"]==1]

,cleantext,label
0,늘 영상만 보다가 처음 댓글 남겨요 큐앤에이 보니까 더 정감가고 친근하네요 다정님,1
2,강아지 오둥이들 키우는 개아범 입니다 항상 유튜브 보면서 힐링하고 있어요 가끔 집...,1
3,라이브못봐서 영상 기다렸어요,1
4,공대생이셨다니 멋져요 다음 라이브는 꼭 시간 맞춰올게요 아숩지뭐에요,1
7,그거 무서워서 못해봤는데 시도해볼게요 이상민이 쓰는거 사서 해봐야겠어요,1
...,...,...
6312,도어알람귀여워요 알콩달콩,1
6313,오 세상 참 좋아졌네요 스마트한 집이라니,1
6314,집이 너무 예뻐요 조명 덕분에 더 분위기 있어 보이네요,1
6315,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요,1


In [ ]:
df['label'] = list_pred
df

,comments,name,time,likes,topic,label
0,늘 영상만 보다가 처음 댓글 남겨요~^^ 큐앤에이 보니까 더 정감가고 친근하네요 다...,맨사언니,2021-04-04T14:15:48Z,1.0,NaN,1
1,"Hi eonni, give english subtitles please🙇",anggi naberia,2021-03-16T22:58:45Z,0.0,NaN,0
2,강아지 오둥이들 키우는 개아범 입니다 ~~~ 항상 유튜브 보면서 힐링하고 있어요~~...,설랜드네 티브이,2021-03-15T19:46:06Z,1.0,NaN,1
3,라이브못봐서 영상 기다렸어요,조서현,2021-03-13T08:59:54Z,1.0,NaN,1
4,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0.0,1.0,1
...,...,...,...,...,...,...
6312,도어알람귀여워요🥰 알콩달콩,박은경,2019-12-19T07:49:07Z,1.0,NaN,1
6313,"오, 세상 참 좋아졌네요 ㅎㅎ<br>스마트한 집이라니..",드림크루의 Hotspot,2019-12-19T05:06:38Z,2.0,NaN,1
6314,집이 너무 예뻐요~!! 조명 덕분에 더 분위기 있어 보이네요💕,예정효,2019-12-19T05:02:05Z,2.0,NaN,1
6315,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요!,김대원,2019-12-19T04:32:38Z,0.0,1.0,1


In [ ]:
Topic0 = ['사진', '식물', '완성', '엄마', '심지', '마무리', '올해', '자수', '베란다', '시간']
Topic1 = ['크리스마스', '선물', '동동', '시간', '사용', '생각', '퇴사', '노트', '사실', '청소']
Topic2 = ['달력', '동동', '아침', '채소', '청소', '시작', '올해', '요거트', '정리', '엄마']
Topic3 = ['커튼', '겨울', '이불', '완성', '토마토', '추천', '팬지', '커버', '요거트', '오리']
topic_list = []
for comments in df['comments']:
  topic = ""
  for word in Topic0:
    if word in comments:
      topic+="0,"
      break
  for word in Topic1:
    if word in comments:
      topic+="1,"
      break
  for word in Topic2:
    if word in comments:
      topic+="2,"
      break
  for word in Topic3:
    if word in comments:
      topic+="3,"
      break
  if len(topic)==0:
    topic_list.append(topic)
  else:
    topic_list.append(topic[:-1])
df['topic'] = topic_list

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df

,comments,name,time,likes,topic,label
0,늘 영상만 보다가 처음 댓글 남겨요~^^ 큐앤에이 보니까 더 정감가고 친근하네요 다...,맨사언니,2021-04-04T14:15:48Z,1.0,,1
1,"Hi eonni, give english subtitles please🙇",anggi naberia,2021-03-16T22:58:45Z,0.0,,0
2,강아지 오둥이들 키우는 개아범 입니다 ~~~ 항상 유튜브 보면서 힐링하고 있어요~~...,설랜드네 티브이,2021-03-15T19:46:06Z,1.0,,1
3,라이브못봐서 영상 기다렸어요,조서현,2021-03-13T08:59:54Z,1.0,,1
4,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0.0,"0,1",1
...,...,...,...,...,...,...
6305,도어알람귀여워요🥰 알콩달콩,박은경,2019-12-19T07:49:07Z,1.0,,1
6306,"오, 세상 참 좋아졌네요 ㅎㅎ<br>스마트한 집이라니..",드림크루의 Hotspot,2019-12-19T05:06:38Z,2.0,,1
6307,집이 너무 예뻐요~!! 조명 덕분에 더 분위기 있어 보이네요💕,예정효,2019-12-19T05:02:05Z,2.0,,1
6308,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요!,김대원,2019-12-19T04:32:38Z,0.0,1,1


In [ ]:
# topic이 없는 comments 삭제
df2 = df.drop(df[df['topic']==""].index)
df2 = df2.reset_index(drop=True)
df2

,comments,name,time,likes,topic,label
0,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0.0,"0,1",1
1,언제 공지했을까요? 함께 하고 싶었어요 !!아쉽 <br>항상 영상 보면 저도 모르...,Čkd buds,2021-03-06T05:25:18Z,2.0,2,1
2,앗 라이브 보려했는데 아쉬워요 🥺 담에도 시간 괜찮으실때 해주세요!! 👏🏻,또*강누나,2021-03-05T05:58:44Z,3.0,"0,1",1
3,다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요! 집안곳곳 다정님이 고민하시고...,백도,2021-03-07T04:23:30Z,1.0,0,1
4,"상부장 위쪽은 실크벽지로 마감되어있고, 욕실 바닥은 흰색과 하늘색 타일을 섞어서 사...",다정하게DAJUNG,2020-10-25T11:19:10Z,0.0,1,0
...,...,...,...,...,...,...
1293,아고 감사합니다 😭 요리는 엄마 어깨 너머로 보고 배우고 있어요 😬,다정하게DAJUNG,2020-05-27T07:03:00Z,0.0,"0,2",1
1294,@다정하게DAJUNG 어제 알려주신대로 잘찿아 구 색맟춰주문했어요.일반 브루스터에...,TheGrldogs,2020-07-10T12:14:17Z,0.0,1,1
1295,양현서 넹넹 그렇게 사용하시면 되어요 😁,다정하게DAJUNG,2020-05-19T10:57:56Z,0.0,1,0
1296,스마트홈이 어려워보였었는데 굉장히 쉽게 사용하는 모습을 보니 한 번 해보고 싶어지네요!,김대원,2019-12-19T04:32:38Z,0.0,1,1


In [ ]:
df2.to_csv("/content/drive/My Drive/DAJUNG_comments.csv", encoding='utf-8-sig', index=False)

In [ ]:
print(df2.groupby("label").size())

label
0     287
1    1011
dtype: int64


In [103]:
# 부정
df2[df2["label"]==0].sort_values(by=['likes'], axis=0, ascending=False)

,comments,name,time,likes,topic,label
165,배워볼까 해서 눌렀는데 &#39; 매일매일 &#39;이 귀에 걸리네요 ㅋㅋㅋㅋㅋㅋㅋ...,DH Min,2020-03-01T11:39:26Z,64.0,"1,2",0
1267,커피머신 캡슐이 재활용이 안된대요.. <br>하루에 한잔씩 한달이면 어마어마한 쓰레...,존좋,2020-02-10T00:56:14Z,35.0,1,0
1114,"오늘도 놀러와 주셔서 감사합니다 🤍<br><a href=""https://www.yo...",다정하게DAJUNG,2020-05-31T11:19:29Z,15.0,0,0
268,영상 메뉴 톱니바퀴 모양을 누르시면 고화질로 감상하실수 있습니다. 늘 감사합니다 :...,다정하게DAJUNG,2020-08-16T11:31:41Z,14.0,2,0
62,"🖤 고민부부 집 꾸미기 vlog 몰아보기: <a href=""https://url....",다정하게DAJUNG,2020-09-06T11:09:12Z,13.0,"0,2,3",0
...,...,...,...,...,...,...
585,저도 에어컨 청소 해야겠어요 ㅎㅎ 날이 꿉꿉해서 틀 때가 된것같아요 😊,다정하게DAJUNG,2021-05-16T15:02:18Z,0.0,"1,2",0
587,정말 너무 싫어요 옷정리 🥲 ㅎㅎㅎ,다정하게DAJUNG,2021-05-16T15:01:54Z,0.0,2,0
593,처음 했을 때 몸살났다 ㅋㅋㅋㅋㅋ 나중에 내가 잘 만들게 되면 하나 선물 해 주지 💛,다정하게DAJUNG,2021-06-06T14:30:20Z,0.0,1,0
597,"안녕하세요, 카메라 색채가 너무 제가 원하는 색채가 나오는 카메라인데 어떤카메라 사...",sijin kim,2021-02-26T15:10:24Z,0.0,1,0


**토픽 라벨링**

In [101]:
topic_n = [0]*4
pos_n = [0]*4
for idx, topic in enumerate(df2['topic']):
  if '0' in topic:
    topic_n[0]+=1
    if df2['label'][idx]==1:
      pos_n[0]+=1
  if '1' in topic:
    topic_n[1]+=1
    if df2['label'][idx]==1:
      pos_n[1]+=1
  if '2' in topic:
    topic_n[2]+=1
    if df2['label'][idx]==1:
      pos_n[2]+=1
  if '3' in topic:
    topic_n[3]+=1
    if df2['label'][idx]==1:
      pos_n[3]+=1

In [102]:
print(topic_n)
print(pos_n)

[467, 599, 496, 275]
[386, 456, 369, 221]


In [107]:
print('Topic 0 댓글 {0:d}개 중 긍정 {1:.2f}, 부정 {2:.2f}'.format(topic_n[0], pos_n[0]/topic_n[0]*100, (1-pos_n[0]/topic_n[0])*100))
print('Topic 1 댓글 {0:d}개 중 긍정 {1:.2f}, 부정 {2:.2f}'.format(topic_n[1], pos_n[1]/topic_n[1]*100, (1-pos_n[1]/topic_n[1])*100))
print('Topic 2 댓글 {0:d}개 중 긍정 {1:.2f}, 부정 {2:.2f}'.format(topic_n[2], pos_n[2]/topic_n[2]*100, (1-pos_n[2]/topic_n[2])*100))
print('Topic 3 댓글 {0:d}개 중 긍정 {1:.2f}, 부정 {2:.2f}'.format(topic_n[3], pos_n[3]/topic_n[3]*100, (1-pos_n[3]/topic_n[3])*100))

Topic 0 댓글 467개 중 긍정 82.66, 부정 17.34
Topic 1 댓글 599개 중 긍정 76.13, 부정 23.87
Topic 2 댓글 496개 중 긍정 74.40, 부정 25.60
Topic 3 댓글 275개 중 긍정 80.36, 부정 19.64
